In [2]:
from tensorflow.examples.tutorials.mnist import input_data

In [3]:
mnist = input_data.read_data_sets("MNIST_data/",one_hot = True)

Extracting MNIST_data/train-images-idx3-ubyte.gz
Extracting MNIST_data/train-labels-idx1-ubyte.gz
Extracting MNIST_data/t10k-images-idx3-ubyte.gz
Extracting MNIST_data/t10k-labels-idx1-ubyte.gz


In [4]:
import tensorflow as tf

In [5]:
x_in = tf.placeholder("float", shape = [None, 784])
y_label = tf.placeholder("float", shape = [None, 10])
w_c1 = tf.Variable(tf.truncated_normal([5,5,1,32],stddev = 0.1)) 
b_c1 = tf.Variable(tf.constant(0.1, shape = [32]))
w_c2 = tf.Variable(tf.truncated_normal([5,5,32,64],stddev = 0.1)) 
b_c2 = tf.Variable(tf.constant(0.1, shape = [64]))
w_f1 = tf.Variable(tf.truncated_normal([3136,1024],stddev = 0.1))
b_f1 = tf.Variable(tf.constant(0.1, shape = [1024]))
w_f2 = tf.Variable(tf.truncated_normal([1024,10],stddev = 0.1))
b_f2 = tf.Variable(tf.constant(0.1, shape = [10]))

In [6]:
x_re = tf.reshape(x_in, [-1,28,28,1])

In [7]:
h_c1 = tf.nn.relu(tf.nn.conv2d(x_re,w_c1,strides = [1,1,1,1], padding = 'SAME') + b_c1)

In [8]:
h_c1p = tf.nn.max_pool(h_c1,ksize=[1, 2, 2, 1], strides = [1,2,2,1], padding = 'SAME')

In [9]:
h_c2 = tf.nn.relu(tf.nn.conv2d(h_c1p,w_c2,strides = [1,1,1,1], padding = 'SAME') + b_c2)

In [10]:
h_c2p = tf.nn.max_pool(h_c2,ksize=[1,2,2,1], strides=[1,2,2,1], padding='SAME')

In [11]:
h_c2p_re = tf.reshape(h_c2p, [-1,3136])

In [12]:
h_fc = tf.nn.relu(tf.matmul(h_c2p_re,w_f1) + b_f1)

In [13]:
dp_p = tf.placeholder("float")
dp = tf.nn.dropout(h_fc, dp_p)

In [14]:
y_out = tf.nn.softmax(tf.matmul(dp,w_f2) + b_f2)

In [15]:
c_entropy = - tf.reduce_sum(y_label * tf.log(y_out))
training_step = tf.train.AdamOptimizer(1e-4).minimize(c_entropy)
an = tf.cast(tf.equal(tf.argmax(y_label,1),tf.argmax(y_out,1)),"float")
correct = tf.reduce_mean(an)

In [16]:
sess = tf.Session()
sess.run(tf.global_variables_initializer())

In [22]:
for t in range(5000):
    batch = mnist.train.next_batch(30)
    if t% 1000 == 0:
        t_a = sess.run(correct,feed_dict={x_in:batch[0], y_label:batch[1], dp_p : 1.0})
        print("step : %d, acc : %g"%(t,t_a))
    sess.run(training_step,feed_dict = {x_in:batch[0], y_label:batch[1], dp_p : 0.7})

step : 0, acc : 0.966667
step : 1000, acc : 0.9
step : 2000, acc : 0.966667
step : 3000, acc : 0.933333
step : 4000, acc : 0.933333


In [23]:
print(sess.run(correct,feed_dict={x_in:mnist.test.images, y_label:mnist.test.labels, dp_p : 1.0}))

0.9862
